In [30]:
!pip install clickhouse_connect faker

In [31]:
import pandas as pd
import numpy as np
from faker import Faker
import random
from datetime import datetime, timedelta
import clickhouse_connect
from dateutil.relativedelta import relativedelta

In [32]:
# Создание клиента для подключения к ClickHouse
client = clickhouse_connect.get_client(host='31.129.97.202', port='8123', user='default', password= '111')

In [33]:
# creating a table in ClickHouse
create_table_sql = """
CREATE TABLE data_mart_with_partition (
    timeseries DateTime,
    Date Date,
    City String,
    Client String,
    Product String,
    metric_1 Float64,
    metric_2 Float64,
    metric_3 Float64,
    metric_4 Float64
)
ENGINE = MergeTree
PARTITION BY toYYYYMM(Date)
ORDER BY (Date, City, Client, Product)
;
"""

client.command('DROP TABLE IF EXISTS data_mart_with_partition')
client.command(create_table_sql)

print('Таблица data_mart_with_partition создана в ClickHouse')

Таблица data_mart_with_partition создана в ClickHouse


In [34]:
# Инициализация Faker для генерации случайных данных
fake = Faker()

In [35]:
def generate_dataframe(num_rows, start_date, end_date):
    print(f"num_rows = {num_rows}. Generate DataFrame for period: {start_date} to {end_date}")

    # Генерация случайных данных для датафрейма
    data = {
        'Date': [fake.date_between_dates(date_start=start_date, date_end=end_date) for _ in range(num_rows)],
        'City': [fake.city() for _ in range(num_rows)],
        'Client': [fake.name() for _ in range(num_rows)],
        'Product': [fake.word() for _ in range(num_rows)],
    }

    # Генерация случайных значений для метрик
    for i in range(1, 5):
        data[f'metric_{i}'] = np.round(np.random.uniform(low=i, high=i*20, size=num_rows)/np.random.uniform(low=1, high=i), 2)

    # Создание датафрейма
    df = pd.DataFrame(data)

    # creating a timeseries field
    df['timeseries'] = pd.to_datetime(df['Date'])

    print('Dataframe сгенерирован')

    # Sending data to ClickHouse
    client.insert_df('data_mart_with_partition', df)

    print('Данные вставлены в таблицу data_mart_with_partition')

In [36]:
# Начальная и конечная дата
start_date = datetime(2024, 1, 1)
end_date = datetime(2024, 2, 29)

# Задаем количество строк в датафрейме
num_rows = 100000

# Вставка N раз по num_rows строк
for i in range(1, 11):
    print(f"*** Вставка данных № {i}")
    generate_dataframe(num_rows, start_date, end_date)

*** Вставка данных № 1
num_rows = 100000. Generate DataFrame for period: 2024-01-01 00:00:00 to 2024-02-29 00:00:00
Dataframe сгенерирован
Данные вставлены в таблицу data_mart_with_partition
*** Вставка данных № 2
num_rows = 100000. Generate DataFrame for period: 2024-01-01 00:00:00 to 2024-02-29 00:00:00
Dataframe сгенерирован
Данные вставлены в таблицу data_mart_with_partition
*** Вставка данных № 3
num_rows = 100000. Generate DataFrame for period: 2024-01-01 00:00:00 to 2024-02-29 00:00:00
Dataframe сгенерирован
Данные вставлены в таблицу data_mart_with_partition
*** Вставка данных № 4
num_rows = 100000. Generate DataFrame for period: 2024-01-01 00:00:00 to 2024-02-29 00:00:00
Dataframe сгенерирован
Данные вставлены в таблицу data_mart_with_partition
*** Вставка данных № 5
num_rows = 100000. Generate DataFrame for period: 2024-01-01 00:00:00 to 2024-02-29 00:00:00
Dataframe сгенерирован
Данные вставлены в таблицу data_mart_with_partition
*** Вставка данных № 6
num_rows = 100000. Gen

In [37]:
# Check count of rows in the table
result_df = client.query_df('SELECT count(1) FROM data_mart_with_partition')
print()
print(result_df)


   count()
0  1000000
